You can find dataset description here https://archive.ics.uci.edu/ml/datasets/Student+Performance. Data is in "Data Folder" name "student.zip"

## How to complete this assignment

First download the data and extract two tables from the link above. Then complete 9 graded tasks below, the score is given in brackets. Finally submit the resulting `.ipynb` file to rs-app Auto-test.
- **Do not delete or rename the variables** given before the inscription '#your code here', they are needed for correct verification.
- **Do not change the code** in the last Notebook cell, it is required for server side.
- Your **Notebook must run completely without errors** in order to be graded! Please check everything before submission by going _Cell_ -> _Run All_

## Here goes!

### Import necessary libraries 

In [4]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

### Read the data

In [5]:
mat = pd.read_csv("student-mat.csv", sep=";")
por = pd.read_csv("student-por.csv", sep=";")

### Look at first five rows of table with students math grades. In all other tasks, unless otherwise stated, we use it.

In [6]:
pd.options.display.max_columns = None
mat.head()

,school,sex,age,address,famsize,Pstatus,Medu,Fedu,Mjob,Fjob,reason,guardian,traveltime,studytime,failures,schoolsup,famsup,paid,activities,nursery,higher,internet,romantic,famrel,freetime,goout,Dalc,Walc,health,absences,G1,G2,G3
0,GP,F,18,U,GT3,A,4,4,at_home,teacher,course,mother,2,2,0,yes,no,no,no,yes,yes,no,no,4,3,4,1,1,3,6,5,6,6
1,GP,F,17,U,GT3,T,1,1,at_home,other,course,father,1,2,0,no,yes,no,no,no,yes,yes,no,5,3,3,1,1,3,4,5,5,6
2,GP,F,15,U,LE3,T,1,1,at_home,other,other,mother,1,2,3,yes,no,yes,no,yes,yes,yes,no,4,3,2,2,3,3,10,7,8,10
3,GP,F,15,U,GT3,T,4,2,health,services,home,mother,1,3,0,no,yes,yes,yes,yes,yes,yes,yes,3,2,2,1,1,5,2,15,14,15
4,GP,F,16,U,GT3,T,3,3,other,other,home,father,1,2,0,no,yes,yes,no,yes,yes,no,no,4,3,2,1,2,5,4,6,10,10


**Task 1 (0.25 points).** Count number of boys and girls in the dataset.

In [7]:
girls, boys = mat['sex'].value_counts()[['F','M']]

**Task 2 (0.25 points).** Count number of 17 year old students having math grade G3 between 10 and 15 inclusively 

In [8]:
students_17_year_old = mat[(mat['age'] == 17) & (mat['G3'].between(10,15))].shape[0]

**Task 3 (0.5 points).** List top-10 students having biggest average grade (G1, G2, G3)  in math. Count number of boys from that list.

In [9]:
mat['avg_grade'] = mat[['G1','G2','G3']].mean(axis=1)
top_10 = mat.sort_values('avg_grade', ascending=False)[:10]
boys_from_top_10 = len(top_10[top_10['sex']=='M'])
mat = mat.drop('avg_grade', axis=1)

In [10]:
top_10 = mat.loc[mat[['G1','G2','G3']].stack().mean(level=0).sort_values(ascending=False).index[0:10]]
boys_from_top_10 = len(top_10[top_10['sex'] == 'M'])

**Task 4 (1 point).** Count mean and standard deviation of absences for students with romantic relationship and access to internet

In [11]:
rom_net = mat[(mat['romantic'] == 'yes') & (mat['internet'] == 'yes')]['absences']
mean, std = rom_net.mean(), rom_net.std()

**Task 5 (1.5 points).** Count number of students for each existing age. Return list with counts sorted by students age in ascending order.

Note: On 0 index in list we have number of 15 year old students, on 1st index number of 16 year old students, ...

In [12]:
student_counts_for_each_age = [len(i[1]) for i in mat.groupby(['age'])]

In [13]:
student_counts_for_each_age = list(mat.groupby('age').size().sort_index().values)

**Task 6* (2 points).** Find most frequent pair (Fjob, Mjob) of students from math grade table.

Note 1: Do not take student into account if at least one of parents job name is "other". <br>
Note 2: Pairs (Fjob: teacher, Mjob: at_home) and (Fjob: at_home, Mjob: teacher) are different and their occurrences should be counted separately.

Hint: groupby (Fjob, Mjob), count rows, descending sort by count, reset_index, take first row values.

In [14]:
job_count = mat[(mat['Fjob'] != 'other') & (mat['Mjob'] != 'other')].groupby(['Fjob', 'Mjob']).size().reset_index().sort_values(by=0, ascending=False)
father, mother = job_count['Fjob'].iloc[0], job_count['Mjob'].iloc[0]

In [15]:
mat_job = mat[(mat['Fjob'] != 'other') & (mat['Mjob'] != 'other')]
jobs = mat_job.groupby(['Fjob','Mjob']).size().reset_index(name='size').sort_values(by='size', ascending=False)
father, mother = jobs.iloc[0,0:2]

**Task 7 (1 point).** Count number of students presenting both in mat and por tables.

_Note 1_: That's a pity that tables haven't foreign keys for joining together. The uniqueness of the student is determined by the uniqueness of the set of all fields except subject fields (G1, G2, G3). <br>
_Note 2_: Finally you'd better to come up with table having all student personal information and non-null grades in math ["G1_mat", "G2_mat", "G3_mat"] and ["G1_por", "G2_por", "G3_por"]. Joined dataframe could be helpful for further usage. <br>
Note 3: probably you would like to use "merge" to avoid duplicating column names from right and left dataframe.

In [16]:
both_present = len(mat.drop(['G1', 'G2', 'G3'], axis=1).merge(por.drop(['G1', 'G2', 'G3'], axis=1), how='inner'))

In [17]:
cols = [x for x in list(mat.columns) if x not in ['G1','G2','G3']]
df = pd.merge(mat, por, how='inner', on=cols, suffixes=('_mat','_por'))
both_present = len(df)

**Task 8 (1.5 points).** Find absolute differences between average mat and por student final grades for each school (GP & MS)

In [18]:
GP_diff, MS_diff = (mat.groupby('school')['G3'].mean() - por.groupby('school')['G3'].mean()).abs()

In [19]:
means = df[['school','G3_mat','G3_por']].groupby('school').mean()
GP_diff, MS_diff = (means['G3_mat'] - means['G3_por']).abs()

**Task 9* (2 points).** Find top-5 linearly dependent pairs of numeric student features from mat table.

_Note 1_: Use absolute Pearson correlation value for measuring the linear relationship between features.

Correlation coefficients are used in statistics to measure how strong a relationship is between two variables. There are several types of correlation coefficients, but the most popular is Pearson’s. It can take value from -1 to 1 inclusive. 
 - A correlation coefficient of 1 means that for every positive increase in one variable, there is a positive increase of a fixed proportion in the other. For example, shoe sizes go up in (almost) perfect correlation with foot length.
 - A correlation coefficient of -1 means that for every positive increase in one variable, there is a negative decrease of a fixed proportion in the other. For example, the amount of gas in a tank decreases in (almost) perfect correlation with speed.
 - Zero means that for every increase of one variable we do not expect always increase or decrease of another variable. It is not a proof, that variables are not related with each other. That only means NO LINEAR relationship between them. Suppose we have variables x and y connected by a sinusoidal law $y=sin(x)$. There is obvious no linear relationship between them (for $x \in [0, \frac{\pi}{2}]$ y increases unevenly ant later for $x \in [\frac{\pi}{2}, \pi]$ y decreases unevenly). Even though they relate to each other by more complex law.
 
More mathematical explanation https://en.wikipedia.org/wiki/Correlation_and_dependence.

In [20]:
# Just a short example
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

x = np.array(np.arange(0, 314, 10)) / 100
y1 = -x + 10
y2 = np.sin(x)

fig, (ax1, ax2) = plt.subplots(2, 1)
ax1.scatter(x, -x + 10)
ax1.set_title("Pearson correlation coefficient for variables x and y=(-x + 10) is %.2f. Negative correlation."
      % np.corrcoef(x, y1)[0][1])
ax1.set_xlabel("X")
ax1.set_ylabel("Y =-X + 10")
ax2.scatter(x, y2)
ax2.set_title("Pearson correlation coefficient for variables x and y=sin(x) is %.2f. No correlation."
      % np.corrcoef(x, y2)[0][1])
ax2.set_xlabel("X")
ax2.set_ylabel("Y =sin(X)")

plt.subplots_adjust(hspace=1.0)
plt.show()

_Note 2_: Use Pandas _.corrcoef_  for calculating Pearson correlation https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.corr.html. Return list of lists of column name pairs by absolute correlation value in descending order. Each pair of names should follow the alphabet order.

In [21]:
corr = mat.corr(method ='pearson').abs()
np.fill_diagonal(corr.values, np.nan)
corr = corr.unstack().sort_values(ascending=False).drop_duplicates().head(5)
top5_correlations = [sorted(corr.index[i]) for i in range(len(corr))]
print(top5_correlations)

[['G2', 'G3'], ['G1', 'G2'], ['G1', 'G3'], ['Dalc', 'Walc'], ['Fedu', 'Medu']]


**Task 10 (bonus no grading).** Plot 5 scatterplots showing relationship between most correlated pairs of variables from the previous task.

<AxesSubplot:xlabel='Fedu', ylabel='Medu'>

In [39]:
fig, axes = plt.subplots(2, 3, figsize=(15, 8))
mat.plot.scatter(x='G2', y='G3', ax=axes[0,0])
mat.plot.scatter(x='G1', y='G2', ax=axes[0,1])
mat.plot.scatter(x='G1', y='G3', ax=axes[0,2])
mat.plot.scatter(x='Dalc', y='Walc', ax=axes[1,0])
mat.plot.scatter(x='Fedu', y='Medu', ax=axes[1,1])

In [576]:
sns.pairplot(mat[['G1','G2','G3','Dalc','Walc','Medu','Fedu']])

### Warning! Do not change anything in this area.

In [577]:
with open('student_answers.txt', 'w') as file:
    file.write(f"{boys} {girls}\n")
    file.write(f"{students_17_year_old}\n")
    file.write(f"{boys_from_top_10}\n")
    file.write(f"{round(mean, 3)} {round(std, 3)}\n")
    file.write(f"{student_counts_for_each_age}\n")
    file.write(f"{father} {mother}\n")
    file.write(f"{both_present}\n")
    file.write(f"{round(GP_diff, 3)} {round(MS_diff, 3)}\n")
    file.write(f"{top5_correlations}")